# Binning of temperature-dependent ARPES data using time-stamped external temperature data
In this example, we pull some temperature-dependent ARPES data from Zenodo, which was recorded as a continuous temperature ramp. We then add the respective temperature information from the respective timestamp/temperature values to the dataframe, and bin the data as function of temperature
For performance reasons, best store the data on a locally attached storage (no network drive). This can also be achieved transparently using the included MirrorUtil class.

In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
import os
import glob

import sed
from sed.dataset import dataset

%matplotlib widget

## Load Data

In [ ]:
dataset.get("TaS2") # Put in Path to a storage of at least 20 GByte free space.
data_path = dataset.dir
scandir, caldir = dataset.subdirs # scandir contains the data, caldir contains the calibration files

# correct timestamps if not correct timezone set
tzoffset = os.path.getmtime(scandir + '/Scan0121_1.h5') - 1594998158.0
if tzoffset:
    for file in glob.glob(scandir +'/*.h5'):
        os.utime(file, (os.path.getmtime(file)-tzoffset, os.path.getmtime(file)-tzoffset))

In [ ]:
# create sed processor using the config file with time-stamps:
sp = sed.SedProcessor(folder=scandir, user_config="../config/mpes_example_config.yaml", system_config={}, time_stamps=True, verbose=True)

In [ ]:
# Apply jittering to X, Y, t, ADC columns.
sp.add_jitter()

In [ ]:
sp.bin_and_load_momentum_calibration(df_partitions=10, plane=33, width=3, apply=True)

In [ ]:
features = np.array([[337., 242.], [289., 327.], [187., 344.], [137., 258.], [189., 161.], [289., 158.], [236.0, 250.0]])
sp.define_features(features=features, rotation_symmetry=6, include_center=True, apply=True)
sp.generate_splinewarp(include_center=True)

In [ ]:
# Adjust pose alignment, using stored distortion correction
sp.pose_adjustment(xtrans=15, ytrans=8, angle=-5, apply=True)

In [ ]:
# Apply stored momentum correction
sp.apply_momentum_correction()

In [ ]:
# Apply stored config momentum calibration
sp.apply_momentum_calibration()

In [ ]:
# Apply stored config energy correction
sp.apply_energy_correction()

In [ ]:
# Load energy calibration EDCs
scans = np.arange(127,136)
voltages = np.arange(21,12,-1)
files = [caldir + r'/Scan' + str(num).zfill(4) + '_1.h5' for num in scans]
sp.load_bias_series(data_files=files, normalize=True, biases=voltages, ranges=[(64000, 76000)])
rg = (65500, 66000)
sp.find_bias_peaks(ranges=rg, ref_id=5, infer_others=True, apply=True)
sp.calibrate_energy_axis(ref_energy=-0.5, energy_scale="kinetic", method="lmfit")

In [ ]:
# Apply stored config energy calibration
#sp.append_energy_axis(bias_voltage=17)
sp.append_energy_axis()

In [ ]:
# add time-stamped temperature data
# either, directly retrieve data from EPICS archiver instance (within FHI network),
#sp.add_time_stamped_data(dest_column="T_B", archiver_channel="trARPES:Carving:TEMP-B")
# or use externally provided timestamp/data pairs
import h5py
with h5py.File(f"{data_path}/temperature_data.h5", "r") as file:
    data = file["temperatures"][()]
    time_stamps = file["timestamps"][()]
sp.add_time_stamped_data(dest_column="sample_temperature", time_stamps=time_stamps, data=data)

In [ ]:
# inspect calibrated event histogram
axes = ['kx', 'ky', 'energy', 'sample_temperature']
ranges = [[-3, 3], [-3, 3], [-6, 2], [10, 300]]
sp.view_event_histogram(dfpid=80, axes=axes, ranges=ranges)

## Define the binning ranges and compute calibrated data volume

In [ ]:
axes = ['kx', 'ky', 'energy', 'sample_temperature']
bins = [100, 100, 300, 100]
ranges = [[-2, 2], [-2, 2], [-6, 2], [20, 270]]
res = sp.compute(bins=bins, axes=axes, ranges=ranges, normalize_to_acquisition_time="sample_temperature")

## Some visualization:

In [ ]:
fig, axs = plt.subplots(4, 1, figsize=(4, 12), constrained_layout=True)
res.loc[{'energy':slice(-.1, 0)}].sum(axis=(2,3)).T.plot(ax=axs[0])
res.loc[{'kx':slice(-.2, .2)}].sum(axis=(0,3)).T.plot(ax=axs[1])
res.loc[{'ky':slice(-.2, .2)}].sum(axis=(1,3)).T.plot(ax=axs[2])
res.loc[{'kx':slice(-.2, .2), 'ky':slice(-.2, .2), 'energy':slice(-2, 0.2)}].sum(axis=(0,1)).plot(ax=axs[3])

In [ ]:
# Inspect effect of histogram normalization
fig, ax = plt.subplots(1,1)
(sp._normalization_histogram/sp._normalization_histogram.sum()).plot(ax=ax)
(sp._binned.sum(axis=(0,1,2))/sp._binned.sum(axis=(0,1,2,3))).plot(ax=ax)
plt.show()

In [ ]:
# Remaining fluctuations are an effect of the varying count rate throughout the scan
plt.figure()
rate, secs = sp.loader.get_count_rate()
plt.plot(secs, rate)

In [ ]:
# Normalize for intensity around the Gamma point
res_norm = res.copy()
res_norm = res_norm/res_norm.loc[{'kx':slice(-.3, .3), 'ky':slice(-.3, .3)}].sum(axis=(0,1,2))

In [ ]:
fig, axs = plt.subplots(4, 1, figsize=(4, 12), constrained_layout=True)
res_norm.loc[{'energy':slice(-.1, 0)}].sum(axis=(2,3)).T.plot(ax=axs[0])
res_norm.loc[{'kx':slice(-.2, .2)}].sum(axis=(0,3)).T.plot(ax=axs[1])
res_norm.loc[{'ky':slice(-.2, .2)}].sum(axis=(1,3)).T.plot(ax=axs[2])
res_norm.loc[{'kx':slice(-.2, .2), 'ky':slice(-.2, .2), 'energy':slice(-2, 0.5)}].sum(axis=(0,1)).plot(ax=axs[3])

In [ ]:
# Lower Hubbard band intensity versus temperature
plt.figure()
res_norm.loc[{'kx':slice(-.2, .2), 'ky':slice(-.2, .2), 'energy':slice(-.6, 0.1)}].sum(axis=(0,1,2)).plot()